## Scraping UN Voting Records

In order to analyze the voting patterns of members and if they match Chinese votes, we are scrapping data from UN libraries. 

Note: This line of work was abandoned as data was found ready for export. 

In [ ]:
import pandas as pd 

# Used to grab URLs and direct people to certain websites. 
import urllib.request
import urllib

# Beautiful Soup used for scrapping from the front end of website. Think scrapping paragraphs to build features. 
import bs4

# Selenium used for webcrawling and jumping to new pages, site navigation. You can also use for scrapping. 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

# Scrapy used for scrapping 
# popular resource, but not used here. 
# import scrapy 

# utilized to conduct parsing of strings. 
import re

import time

In [ ]:
options = webdriver.ChromeOptions()

options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
driver = webdriver.Chrome(service=Service(executable_path=ChromeDriverManager().install()), options=options)
action = ActionChains(driver)
# navigate to site 
driver.get('https://digitallibrary.un.org/search?ln=en&cc=Voting+Data&p=&f=&rm=&ln=en&sf=&so=d&rg=50&c=Voting+Data&c=&of=hb&fti=0&fti=0')

In [ ]:
# create dataframe of records to store information
df = pd.DataFrame(columns=['link', 'year'])

In [ ]:
def process_page(year): 
    
    # create temp listing of links to process for this page
    links = driver.find_elements("class name", value="result-title")

    df_temp = pd.DataFrame({'link' :[x.find_element('xpath', ".//a").get_attribute('href') for x in links], 'year': year})

    df_new = df.append(df_temp, ignore_index=True)
    
    return df_new

In [ ]:
# process each page, and move to the next 
# sort by year first to allow for all decision navigation

for year in range(2020, 2022 + 1): 
    # query for that year (to ensure below 500 query limit)
    ele_year = driver.find_element("xpath", value="//input[@aria-label='" + str(year) + "'][contains(@id, 'desktopcheckbox1-fct')]")
    action.click(on_element=ele_year).perform()
    
    time.sleep(1)
    
    # process all yearly data 
    # start with links on first page, then move on to other records if they can be found
    df = process_page(year)
    
    time.sleep(1)
        
    # check for next page links
    cont = True
    while cont: 
        try: 
            next_page = driver.find_element('xpath', value="//a//img[@aria-label='Next page']")
            action.click(on_element=next_page).perform()

            time.sleep(1)

            df = process_page(year)       
        except: 
            cont = False
            print("all pages viewed for " + str(year))
        
    # uncheck this year's query 
    
    time.sleep(1)
    
    ele_year = driver.find_element("xpath", value="//input[@aria-label='" + str(year) + "'][contains(@id, 'desktopcheckbox1-fct')]")
    action.click(on_element=ele_year).perform()